In [1]:
from typing import List

import torch
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/Salary_dataset.csv")

df = df.drop(columns=['Unnamed: 0'])

df.head(5)

,YearsExperience,Salary
0,1.2,39344.0
1,1.4,46206.0
2,1.6,37732.0
3,2.1,43526.0
4,2.3,39892.0


In [3]:
class SalaryDataset(Dataset):
    def __init__(self, X, y):
        self.features = torch.tensor(X).to(torch.float32)
        self.labels = torch.tensor(y).to(torch.float32)
    
    def __getitem__(self, index):
        feature_item = self.features[index]
        label_item = self.labels[index]

        return feature_item, label_item
    
    def __len__(self):
        return self.features.shape[0]

In [4]:
def train_test_split(X: np.array, y: np.array, test_ratio: float = 0.2) -> List[List]:
    """
    This function splits a dataset into train and test bits
    """
    train_ratio = 1 - test_ratio
    length_of_dataset = len(X)

    train_split = int(length_of_dataset * train_ratio)

    X_train = X[: train_split]
    y_train = y[: train_split]

    X_test = X[train_split: ]
    y_test = X[train_split: ]

    return X_train, y_train, X_test, y_test

In [5]:
X = df['YearsExperience'].values
y = df['Salary'].values

In [6]:
X_train, y_train, X_test, y_test = train_test_split(X, y)

X_train, y_train, X_test, y_test

(array([1.2, 1.4, 1.6, 2.1, 2.3, 3. , 3.1, 3.3, 3.3, 3.8, 4. , 4.1, 4.1,
        4.2, 4.6, 5. , 5.2, 5.4, 6. , 6.1, 6.9, 7.2, 8. , 8.3]),
 array([ 39344.,  46206.,  37732.,  43526.,  39892.,  56643.,  60151.,
         54446.,  64446.,  57190.,  63219.,  55795.,  56958.,  57082.,
         61112.,  67939.,  66030.,  83089.,  81364.,  93941.,  91739.,
         98274., 101303., 113813.]),
 array([ 8.8,  9.1,  9.6,  9.7, 10.4, 10.6]),
 array([ 8.8,  9.1,  9.6,  9.7, 10.4, 10.6]))

In [7]:
train_dataset = SalaryDataset(X=X_train, y=y_train)
test_dataset = SalaryDataset(X=X_test, y=y_test)

In [8]:
# Loading Dataloaders

train_dataloader = DataLoader(
    dataset=train_dataset, 
    batch_size=2,  # You can play around with this batch size
    shuffle=True, 
    num_workers=0, 
    drop_last=True, # If the last batch is not a multiple of 4 then it will remove it (Uneven batch sizes can cause descripensies in our model)
)

test_dataloader = DataLoader(
    dataset=test_dataset, 
    batch_size=2, 
    shuffle=True, 
    drop_last=True, 
    num_workers=0
)

In [12]:
import torch
import torch.nn as nn
from typing import List

class NeuralNetworkCPU(nn.Module):
    def __init__(self, num_inputs: int, num_outputs: int = 1) -> None:
        super(NeuralNetworkCPU, self).__init__()

        self.layers = nn.Sequential(
            # 1st Hidden Layer
            nn.Linear(num_inputs, 100), 
            nn.ReLU(), 

            # 2nd Hidden Layer
            nn.Linear(100, 200), 
            nn.ReLU(), 

            # 3rd Hidden Layer
            nn.Linear(200, 250), 
            nn.ReLU(), 

            # 4th Hidden Layer
            nn.Linear(250, 200), 
            nn.ReLU(), 

            # 5th Hidden Layer
            nn.Linear(200, 100), 
            nn.ReLU(), 

            # 6th Hidden Layer
            nn.Linear(100, 50), 
            nn.ReLU(), 

            # Output Layer
            nn.Linear(50, num_outputs)
        )

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        """
        This function performs a forward pass inference of the model 
        """
        logits = self.layers(input)
        return logits


In [14]:
import torch.nn.functional as F

# Initialize model and optimizer
model = NeuralNetworkCPU(num_inputs=1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)  # A more reasonable learning rate

# Training loop
for epoch in range(1000):
    model.train()  # Set model to training mode
    
    # Iterate over batches in the DataLoader
    for index, (features, labels) in enumerate(train_dataloader):
        
        # Ensure the input has the correct shape (batch_size, 1)
        features = features.view(-1, 1)  # Reshaping to (batch_size, 1)
        
        # Ensure the input and labels are not NaN or Inf
        assert torch.isnan(features).sum() == 0, "Input contains NaN"
        assert torch.isinf(features).sum() == 0, "Input contains Inf"
        assert torch.isnan(labels).sum() == 0, "Labels contain NaN"
        assert torch.isinf(labels).sum() == 0, "Labels contain Inf"

        # Forward pass
        logits = model(features)

        # Compute loss
        loss = F.mse_loss(logits, labels)
        
        # Check if the loss is NaN
        if torch.isnan(loss).sum() > 0:
            print(f"NaN detected at epoch {epoch+1}, batch {index+1}")
            break
        
        # Backpropagation and optimization
        optimizer.zero_grad()  # Zero gradients
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model weights

    # Print progress every epoch
    print(f"Epoch: {epoch+1:03d}/1000 | Train loss: {loss:.4f}")

    # If NaN loss detected, break the outer loop
    if torch.isnan(loss).sum() > 0:
        print("Training stopped due to NaN loss")
        break


NaN detected at epoch 1, batch 4
Epoch: 001/1000 | Train loss: nan
Training stopped due to NaN loss


C:\Users\athar\AppData\Local\Temp\ipykernel_16400\3862159806.py:27: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(logits, labels)
